In [1]:
from stravaio import StravaIO
from stravaio import strava_oauth2
import requests

import json
import pandas as pd
import numpy as np

from datetime import datetime
from datetime import timedelta


with open('strava_ids.json','r') as f:
    strava_data = json.loads(f.read())

strava_auth_data = strava_oauth2(strava_data['client_id'], strava_data['client_secret'])
strava_client = StravaIO(strava_auth_data['access_token'])    

2021-06-30 17:50:28.915 | INFO     | stravaio:strava_oauth2:343 - serving at port 8000
2021-06-30 17:50:31.253 | DEBUG    | stravaio:run_server_and_wait_for_token:397 - code: 66044af687b605389c234afbca529489ea138c62
2021-06-30 17:50:31.777 | DEBUG    | stravaio:run_server_and_wait_for_token:406 - Authorized athlete: 6472d7dbe2f70e7fc5ff791b59faa76b95208590


In [2]:
activity_id = 5518106799
headers = {"Authorization": "Bearer {access_token}".format(access_token = strava_auth_data['access_token'])}
activity_url = 'https://www.strava.com/api/v3/activities/{activity_id}?include_all_efforts'.format(activity_id = activity_id)
response = requests.get(activity_url,headers=headers)
activity_dict = json.loads(response.text)

#distance data stream
url = 'https://www.strava.com/api/v3/activities/{activity_id}/streams?'.format(activity_id = activity_id)
response = requests.get(url,headers=headers)
#print(response)
#print(response.text)
distance_dict = json.loads(response.text[1:len(response.text)-1])
strava_df = pd.DataFrame(distance_dict['data'], columns =['distance'])

#time data stream
url = 'https://www.strava.com/api/v3/activities/{activity_id}/streams?&type=time&series_type=time'.format(activity_id = activity_id)
response = requests.get(url,headers=headers)
#print(response)
time_dict = json.loads(response.text[1:len(response.text)-1])
strava_df['time'] = time_dict['data']

In [3]:
strava_df['time_diff'] = strava_df['time'].shift(-1) - strava_df['time']
strava_df['dist_diff'] = strava_df['distance'].shift(-1) - strava_df['distance']

start_time_str = activity_dict['start_date_local']
start_time = datetime.strptime(start_time_str, '%Y-%m-%dT%H:%M:%SZ')

total_dist = strava_df.iloc[0].distance

delta_dist = delta_time = i = 0
elapsed_time = start_time

strava_df['pace'] = np.nan
strava_df['total_distance'] = np.nan
strava_df['timestamp'] = np.nan

i = 0
for idx, row in strava_df.iterrows():
    delta_dist += row['dist_diff']
    delta_time += row['time_diff']
    if delta_dist >= 100:
        elapsed_time += timedelta(seconds=delta_time)
        strava_df.loc[idx, 'timestamp'] = elapsed_time
        strava_df.loc[idx, 'pace'] = delta_time/60
        total_dist += delta_dist
        strava_df.loc[idx, 'total_distance'] = total_dist    
        i += 1
        delta_dist = delta_time = 0
       
strava_df = strava_df.dropna()
print(i)  

29


In [4]:
from oauth_server import OAuth2Server
import fitbit
with open('fitbit_ids.json','r') as f:
    fitbit_data = json.loads(f.read())
fitbit_oauthserver = OAuth2Server(fitbit_data['client_id'], fitbit_data['client_secret'])
fitbit_oauthserver.browser_authorize()

[30/Jun/2021:17:50:41] ENGINE Listening for SIGTERM.
[30/Jun/2021:17:50:41] ENGINE Listening for SIGHUP.
[30/Jun/2021:17:50:41] ENGINE Listening for SIGUSR1.
[30/Jun/2021:17:50:41] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[30/Jun/2021:17:50:41] ENGINE Started monitor thread 'Autoreloader'.
[30/Jun/2021:17:50:41] ENGINE Serving on http://127.0.0.1:8080
[30/Jun/2021:17:50:41] ENGINE Bus STARTED


127.0.0.1 - - [30/Jun/2021:17:50:43] "GET /?code=a2d7d3713be81e19a98010e775fee89a0cb4a143&state=r88GqSNrUVid5vpZKsFXYZy0sdxwDR HTTP/1.1" 200 122 "" "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.90 Safari/537.36"


[30/Jun/2021:17:50:44] ENGINE Bus STOPPING
[30/Jun/2021:17:50:44] ENGINE HTTP Server cherrypy._cpwsgi_server.CPWSGIServer(('127.0.0.1', 8080)) shut down
[30/Jun/2021:17:50:44] ENGINE Stopped thread 'Autoreloader'.
[30/Jun/2021:17:50:44] ENGINE Bus STOPPED
[30/Jun/2021:17:50:44] ENGINE Bus EXITING
[30/Jun/2021:17:50:44] ENGINE Bus EXITED
[30/Jun/2021:17:50:44] ENGINE Waiting for child threads to terminate...


In [5]:
client = fitbit.Fitbit(fitbit_data['client_id'], fitbit_data['client_secret'],
         access_token = fitbit_oauthserver.fitbit.client.session.token['access_token'],
         refresh_token = fitbit_oauthserver.fitbit.client.session.token['refresh_token'])

heart_url =  "https://api.fitbit.com/1.2/user/-/activities/heart/date/{date}/1d/1sec.json".format(date = start_time.strftime("%Y-%m-%d"))
fitbit_hr_data = client.make_request(heart_url)
heart_data = fitbit_hr_data['activities-heart-intraday']

end_time = start_time + timedelta(seconds = activity_dict['elapsed_time'])  

hb_df = pd.DataFrame(heart_data['dataset'])
hb_df['time'] = pd.to_datetime(hb_df['time'])
hb_df['time'] = hb_df['time'].apply(lambda x: x.replace(year=start_time.year, month=start_time.month, day = start_time.day))
hb_df = hb_df[(hb_df['time'] > start_time) & (hb_df['time'] < end_time )]


In [6]:
from bokeh.plotting import figure, output_file, show
from bokeh.models import LinearAxis, Range1d
from bokeh.models import HoverTool
from bokeh.models import ColumnDataSource

from math import pi
chart_red = '#f45666'
chart_blue = '#4292c6'

chart_tools="tap, crosshair, wheel_zoom,zoom_in,zoom_out"



p = figure(title="swimming pace vs heart beat", 
           x_axis_type="datetime", x_axis_label='time', 
           y_axis_label = 'pace [min/100m]', y_range=(2, 8),
           toolbar_location="below",tools=chart_tools)

p.yaxis.axis_line_color = chart_blue
p.yaxis.major_tick_line_color = chart_blue
p.yaxis.minor_tick_line_color = chart_blue


p.add_tools(HoverTool(
    tooltips=[('time', '$x{%H:%M:%S}'),
              ("value", "$y{0.00}")],
    formatters={'$x':"datetime"}
))

#pace
p.line(source = ColumnDataSource(strava_df),  x = 'timestamp', y = 'pace', color = chart_blue)

#heartbeat
p.extra_y_ranges = {"heart_beat": Range1d(start=60, end=150),}

# axis_label_text_color or major_label_text_color
p.add_layout(LinearAxis(y_range_name="heart_beat", 
                        axis_label="hear beat [bpm]", 
                        axis_line_color = chart_red,
                        major_tick_line_color = chart_red,
                        minor_tick_line_color = chart_red),
                        'right')

p.scatter(source = ColumnDataSource(hb_df),  x = 'time', y = 'value', color = chart_red, y_range_name="heart_beat")

output_file("pace_vs_heartbit.html", title="swimming pace vs heartbeat")
show(p)  